In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.utils import np_utils
np.random.seed(10)
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers
from keras.regularizers import l2

In [64]:
df = pd.read_csv('./Pokemon.csv')
#df['Type 1'].unique()
df.drop(df[(df['Type 1'] =='Ground') | 
           (df['Type 1'] =='Ghost')  |
           (df['Type 1'] =='Dragon') |
           (df['Type 1'] =='Dark') |
           (df['Type 1'] =='Poison') |
           (df['Type 1'] =='Steel') |
           (df['Type 1'] =='Fighting') |
           (df['Type 1'] =='Ice') |
           (df['Type 1'] =='Fairy') |
           (df['Type 1'] =='Flying') ].index,axis=0,inplace=True)

In [65]:
df['Type 1'].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Name: Type 1, dtype: int64

In [66]:
def food_map(df):
    if df['Type 1'] == 'Water':
        return 0
    elif df['Type 1'] == 'Normal':
        return 1
    elif df['Type 1'] == 'Grass':
        return 2
    elif df['Type 1'] == 'Bug':
        return 3
    elif df['Type 1'] == 'Psychic':
        return 4
    elif df['Type 1'] == 'Fire':
        return 5
    elif df['Type 1'] == 'Electric':
        return 6
    elif df['Type 1'] == 'Rock':
        return 7
df['type'] = df.apply(food_map,axis = 'columns')
df

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,type
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,2
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,2
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,2
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,2
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,5
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False,5
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False,5
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False,5
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False,5
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False,0


In [67]:
df.sample(frac=1).reset_index(drop=True)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,type
0,540,Sewaddle,Bug,Grass,310,45,53,70,40,60,42,5,False,3
1,392,Infernape,Fire,Fighting,534,76,104,71,104,71,108,4,False,5
2,480,Uxie,Psychic,NaN,580,75,75,130,75,130,95,4,True,4
3,319,Sharpedo,Water,Dark,460,70,120,40,95,40,95,3,False,0
4,54,Psyduck,Water,NaN,320,50,52,48,65,50,55,1,False,0
5,193,Yanma,Bug,Flying,390,65,65,45,75,45,95,2,False,3
6,667,Litleo,Fire,Normal,369,62,50,58,73,54,72,6,False,5
7,432,Purugly,Normal,NaN,452,71,82,64,64,59,112,4,False,1
8,243,Raikou,Electric,NaN,580,90,85,75,115,100,115,2,True,6
9,547,Whimsicott,Grass,Fairy,480,60,67,85,77,75,116,5,False,2


In [68]:
train = df.iloc[0:500]
test = df.iloc[500:800]

a = train['type']
train_label = np.array(a)

a = test['type']
test_label = np.array(a)

In [69]:
Total_matrix_water =   np.array(train['Total'])
HP_matrix_water =      np.array(train['HP'])
Attack_matrix_water =  np.array(train['Attack'])
Defense_matrix_water = np.array(train['Defense'])
SpAtk_matrix_water =   np.array(train['Sp. Atk'])
SpDef_matrix_water =   np.array(train['Sp. Def'])
Speed_matrix_water =   np.array(train['Speed'])

train_data = np.vstack((Total_matrix_water,
                            HP_matrix_water,
                            Attack_matrix_water,
                            Defense_matrix_water,
                            SpAtk_matrix_water,
                            SpDef_matrix_water,
                            Speed_matrix_water)).T

train_data.shape

(500, 7)

In [70]:
Total_matrix_water =   np.array(test['Total'])
HP_matrix_water =      np.array(test['HP'])
Attack_matrix_water =  np.array(test['Attack'])
Defense_matrix_water = np.array(test['Defense'])
SpAtk_matrix_water =   np.array(test['Sp. Atk'])
SpDef_matrix_water =   np.array(test['Sp. Def'])
Speed_matrix_water =   np.array(test['Speed'])

test_data = np.vstack((Total_matrix_water,
                            HP_matrix_water,
                            Attack_matrix_water,
                            Defense_matrix_water,
                            SpAtk_matrix_water,
                            SpDef_matrix_water,
                            Speed_matrix_water)).T

test_data.shape

(46, 7)

In [71]:
test_data[0]

array([580,  91,  72,  90, 129,  90, 108], dtype=int64)

In [72]:
train_label_onehot = np_utils.to_categorical(train_label)
test_label_onehot = np_utils.to_categorical(test_label)

In [73]:
test_label_onehot.shape

(46, 8)

In [74]:
train_label_onehot.shape

(500, 8)

In [75]:
acc = []
j = 100
#for i in range(5,50,50):
model = Sequential()
    
    
#輸入層：7, 隱藏層：50，輸出層：2
model.add(Dense(units=j,input_dim=7,kernel_regularizer=l2(0.01),activation='relu'))
model.add(Dropout(0.2))
    
model.add(Dense(units=j,kernel_regularizer=l2(0.01),activation='relu'))
model.add(Dropout(0.2))
    
model.add(Dense(units=j,kernel_regularizer=l2(0.01),activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=j,kernel_regularizer=l2(0.01),activation='relu'))
model.add(Dropout(0.2))
   
    
model.add(Dense(units=8,
                    kernel_initializer='normal',
                    activation='softmax'))
    #定義訓練方式
adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',
                  optimizer=adam, metrics=['accuracy'])
train_history =model.fit(x=train_data,
                         y=train_label_onehot,
                         validation_split=0.1, 
                         epochs=10000,
                         batch_size=15,
                         verbose=2)

#評估準確率
scores = model.evaluate(test_data, test_label_onehot)
print('\n準確率=',scores[1])
acc.append(scores[1])

Train on 450 samples, validate on 50 samples
Epoch 1/10000
 - 1s - loss: 14.0784 - acc: 0.1356 - val_loss: 10.3283 - val_acc: 0.1000
Epoch 2/10000
 - 0s - loss: 13.0430 - acc: 0.1378 - val_loss: 7.8894 - val_acc: 0.0200
Epoch 3/10000
 - 0s - loss: 12.0652 - acc: 0.1244 - val_loss: 7.1976 - val_acc: 0.2000
Epoch 4/10000
 - 0s - loss: 10.6527 - acc: 0.1667 - val_loss: 6.4634 - val_acc: 0.1600
Epoch 5/10000
 - 0s - loss: 10.7573 - acc: 0.1289 - val_loss: 6.2154 - val_acc: 0.1600
Epoch 6/10000
 - 0s - loss: 9.7305 - acc: 0.1578 - val_loss: 6.1346 - val_acc: 0.1600
Epoch 7/10000
 - 0s - loss: 9.8257 - acc: 0.1089 - val_loss: 5.9994 - val_acc: 0.1400
Epoch 8/10000
 - 0s - loss: 8.6190 - acc: 0.1600 - val_loss: 5.6328 - val_acc: 0.1600
Epoch 9/10000
 - 0s - loss: 8.3880 - acc: 0.1622 - val_loss: 5.5363 - val_acc: 0.0800
Epoch 10/10000
 - 0s - loss: 7.6010 - acc: 0.1400 - val_loss: 5.3593 - val_acc: 0.1000
Epoch 11/10000
 - 0s - loss: 7.5128 - acc: 0.1533 - val_loss: 5.2078 - val_acc: 0.1200
E

Epoch 95/10000
 - 0s - loss: 3.9752 - acc: 0.1889 - val_loss: 3.9867 - val_acc: 0.1200
Epoch 96/10000
 - 0s - loss: 3.9647 - acc: 0.1844 - val_loss: 3.9811 - val_acc: 0.1200
Epoch 97/10000
 - 0s - loss: 3.9108 - acc: 0.2178 - val_loss: 3.9737 - val_acc: 0.1200
Epoch 98/10000
 - 0s - loss: 3.8990 - acc: 0.2378 - val_loss: 3.9616 - val_acc: 0.1200
Epoch 99/10000
 - 0s - loss: 3.9091 - acc: 0.2067 - val_loss: 3.9475 - val_acc: 0.1400
Epoch 100/10000
 - 0s - loss: 3.9464 - acc: 0.2044 - val_loss: 3.9370 - val_acc: 0.1400
Epoch 101/10000
 - 0s - loss: 3.8831 - acc: 0.2111 - val_loss: 3.9323 - val_acc: 0.1400
Epoch 102/10000
 - 0s - loss: 3.8706 - acc: 0.2378 - val_loss: 3.9296 - val_acc: 0.1400
Epoch 103/10000
 - 0s - loss: 3.8848 - acc: 0.2222 - val_loss: 3.9170 - val_acc: 0.1200
Epoch 104/10000
 - 0s - loss: 3.8744 - acc: 0.1867 - val_loss: 3.9085 - val_acc: 0.1200
Epoch 105/10000
 - 0s - loss: 3.8991 - acc: 0.2267 - val_loss: 3.8987 - val_acc: 0.1600
Epoch 106/10000
 - 0s - loss: 3.8478 

 - 0s - loss: 3.0656 - acc: 0.2511 - val_loss: 3.1251 - val_acc: 0.2000
Epoch 189/10000
 - 0s - loss: 3.0758 - acc: 0.2733 - val_loss: 3.1205 - val_acc: 0.2400
Epoch 190/10000
 - 0s - loss: 3.0181 - acc: 0.2933 - val_loss: 3.1104 - val_acc: 0.2200
Epoch 191/10000
 - 0s - loss: 3.0593 - acc: 0.2378 - val_loss: 3.1043 - val_acc: 0.2400
Epoch 192/10000
 - 0s - loss: 3.0302 - acc: 0.2733 - val_loss: 3.0908 - val_acc: 0.2400
Epoch 193/10000
 - 0s - loss: 3.0635 - acc: 0.2444 - val_loss: 3.0867 - val_acc: 0.2400
Epoch 194/10000
 - 0s - loss: 2.9975 - acc: 0.2778 - val_loss: 3.0840 - val_acc: 0.2600
Epoch 195/10000
 - 0s - loss: 3.0292 - acc: 0.2667 - val_loss: 3.0723 - val_acc: 0.2600
Epoch 196/10000
 - 0s - loss: 2.9952 - acc: 0.3022 - val_loss: 3.0620 - val_acc: 0.3000
Epoch 197/10000
 - 0s - loss: 3.0102 - acc: 0.2511 - val_loss: 3.0655 - val_acc: 0.2800
Epoch 198/10000
 - 0s - loss: 2.9736 - acc: 0.2756 - val_loss: 3.0573 - val_acc: 0.2800
Epoch 199/10000
 - 0s - loss: 2.9891 - acc: 0.27

Epoch 282/10000
 - 0s - loss: 2.3940 - acc: 0.3222 - val_loss: 2.5835 - val_acc: 0.2000
Epoch 283/10000
 - 0s - loss: 2.3970 - acc: 0.3044 - val_loss: 2.5744 - val_acc: 0.2200
Epoch 284/10000
 - 0s - loss: 2.3760 - acc: 0.3111 - val_loss: 2.5475 - val_acc: 0.2200
Epoch 285/10000
 - 0s - loss: 2.3770 - acc: 0.3333 - val_loss: 2.5288 - val_acc: 0.2800
Epoch 286/10000
 - 0s - loss: 2.3989 - acc: 0.3244 - val_loss: 2.5577 - val_acc: 0.2200
Epoch 287/10000
 - 0s - loss: 2.4086 - acc: 0.3156 - val_loss: 2.5366 - val_acc: 0.2200
Epoch 288/10000
 - 0s - loss: 2.3693 - acc: 0.3400 - val_loss: 2.5400 - val_acc: 0.1800
Epoch 289/10000
 - 0s - loss: 2.3500 - acc: 0.3311 - val_loss: 2.5313 - val_acc: 0.2000
Epoch 290/10000
 - 0s - loss: 2.3753 - acc: 0.3133 - val_loss: 2.5287 - val_acc: 0.2200
Epoch 291/10000
 - 0s - loss: 2.3530 - acc: 0.3333 - val_loss: 2.5213 - val_acc: 0.2400
Epoch 292/10000
 - 0s - loss: 2.3617 - acc: 0.3533 - val_loss: 2.5006 - val_acc: 0.2200
Epoch 293/10000
 - 0s - loss: 2.

 - 0s - loss: 2.0395 - acc: 0.3733 - val_loss: 2.2929 - val_acc: 0.2400
Epoch 376/10000
 - 0s - loss: 2.0038 - acc: 0.3778 - val_loss: 2.2927 - val_acc: 0.2400
Epoch 377/10000
 - 0s - loss: 2.0109 - acc: 0.3978 - val_loss: 2.2876 - val_acc: 0.2400
Epoch 378/10000
 - 0s - loss: 2.0339 - acc: 0.3667 - val_loss: 2.3131 - val_acc: 0.2000
Epoch 379/10000
 - 0s - loss: 2.0286 - acc: 0.3711 - val_loss: 2.3002 - val_acc: 0.2400
Epoch 380/10000
 - 0s - loss: 2.0217 - acc: 0.3733 - val_loss: 2.3024 - val_acc: 0.2200
Epoch 381/10000
 - 0s - loss: 2.0128 - acc: 0.3800 - val_loss: 2.2989 - val_acc: 0.2200
Epoch 382/10000
 - 0s - loss: 2.0063 - acc: 0.3556 - val_loss: 2.2843 - val_acc: 0.2200
Epoch 383/10000
 - 0s - loss: 1.9585 - acc: 0.3933 - val_loss: 2.2991 - val_acc: 0.2400
Epoch 384/10000
 - 0s - loss: 2.0194 - acc: 0.3578 - val_loss: 2.2964 - val_acc: 0.2400
Epoch 385/10000
 - 0s - loss: 1.9936 - acc: 0.3689 - val_loss: 2.3145 - val_acc: 0.2400
Epoch 386/10000
 - 0s - loss: 2.0021 - acc: 0.39

...

 - 0s - loss: 0.8679 - acc: 0.7800 - val_loss: 3.8776 - val_acc: 0.2400
Epoch 9544/10000
 - 0s - loss: 0.8526 - acc: 0.7889 - val_loss: 3.8612 - val_acc: 0.2800
Epoch 9545/10000
 - 0s - loss: 0.8859 - acc: 0.7733 - val_loss: 3.8157 - val_acc: 0.2400
Epoch 9546/10000
 - 0s - loss: 0.8245 - acc: 0.8222 - val_loss: 3.6953 - val_acc: 0.2600
Epoch 9547/10000
 - 0s - loss: 0.8523 - acc: 0.7933 - val_loss: 3.7532 - val_acc: 0.2400
Epoch 9548/10000
 - 0s - loss: 0.8117 - acc: 0.8356 - val_loss: 3.8582 - val_acc: 0.2600
Epoch 9549/10000
 - 0s - loss: 0.8647 - acc: 0.8000 - val_loss: 3.7881 - val_acc: 0.2200
Epoch 9550/10000
 - 0s - loss: 0.8500 - acc: 0.8022 - val_loss: 3.8070 - val_acc: 0.2600
Epoch 9551/10000
 - 0s - loss: 0.8392 - acc: 0.8133 - val_loss: 3.6985 - val_acc: 0.2400
Epoch 9552/10000
 - 0s - loss: 0.8454 - acc: 0.7822 - val_loss: 3.8007 - val_acc: 0.2400
Epoch 9553/10000
 - 0s - loss: 0.8788 - acc: 0.8022 - val_loss: 3.7520 - val_acc: 0.2400
Epoch 9554/10000
 - 0s - loss: 0.8542 

Epoch 9636/10000
 - 0s - loss: 0.8455 - acc: 0.7800 - val_loss: 3.8098 - val_acc: 0.2400
Epoch 9637/10000
 - 0s - loss: 0.8381 - acc: 0.8111 - val_loss: 3.8375 - val_acc: 0.2400
Epoch 9638/10000
 - 0s - loss: 0.7867 - acc: 0.8400 - val_loss: 3.7858 - val_acc: 0.2400
Epoch 9639/10000
 - 0s - loss: 0.8592 - acc: 0.7978 - val_loss: 3.7626 - val_acc: 0.2400
Epoch 9640/10000
 - 0s - loss: 0.8728 - acc: 0.7733 - val_loss: 3.9128 - val_acc: 0.2600
Epoch 9641/10000
 - 0s - loss: 0.7866 - acc: 0.8267 - val_loss: 3.8358 - val_acc: 0.2400
Epoch 9642/10000
 - 0s - loss: 0.8418 - acc: 0.7844 - val_loss: 3.7831 - val_acc: 0.2600
Epoch 9643/10000
 - 0s - loss: 0.8464 - acc: 0.8000 - val_loss: 3.7681 - val_acc: 0.2800
Epoch 9644/10000
 - 0s - loss: 0.8548 - acc: 0.8000 - val_loss: 3.9990 - val_acc: 0.2200
Epoch 9645/10000
 - 0s - loss: 0.8302 - acc: 0.8000 - val_loss: 3.7234 - val_acc: 0.2800
Epoch 9646/10000
 - 0s - loss: 0.8021 - acc: 0.8133 - val_loss: 3.7208 - val_acc: 0.2600
Epoch 9647/10000
 - 0

 - 0s - loss: 0.8547 - acc: 0.7911 - val_loss: 3.7744 - val_acc: 0.3000
Epoch 9729/10000
 - 0s - loss: 0.8458 - acc: 0.7911 - val_loss: 3.7333 - val_acc: 0.2400
Epoch 9730/10000
 - 0s - loss: 0.8382 - acc: 0.8156 - val_loss: 3.6419 - val_acc: 0.3000
Epoch 9731/10000
 - 0s - loss: 0.8101 - acc: 0.7956 - val_loss: 3.6899 - val_acc: 0.2600
Epoch 9732/10000
 - 0s - loss: 0.7881 - acc: 0.8267 - val_loss: 3.7807 - val_acc: 0.2600
Epoch 9733/10000
 - 0s - loss: 0.8070 - acc: 0.8156 - val_loss: 3.7652 - val_acc: 0.2600
Epoch 9734/10000
 - 0s - loss: 0.8283 - acc: 0.7978 - val_loss: 3.6341 - val_acc: 0.2800
Epoch 9735/10000
 - 0s - loss: 0.8126 - acc: 0.8067 - val_loss: 3.7804 - val_acc: 0.2800
Epoch 9736/10000
 - 0s - loss: 0.8265 - acc: 0.8244 - val_loss: 3.7113 - val_acc: 0.2600
Epoch 9737/10000
 - 0s - loss: 0.8763 - acc: 0.7822 - val_loss: 3.7742 - val_acc: 0.3000
Epoch 9738/10000
 - 0s - loss: 0.8235 - acc: 0.8200 - val_loss: 3.7350 - val_acc: 0.2800
Epoch 9739/10000
 - 0s - loss: 0.8761 

Epoch 9821/10000
 - 0s - loss: 0.8401 - acc: 0.8000 - val_loss: 3.8599 - val_acc: 0.2800
Epoch 9822/10000
 - 0s - loss: 0.8280 - acc: 0.8044 - val_loss: 3.8220 - val_acc: 0.2800
Epoch 9823/10000
 - 0s - loss: 0.8732 - acc: 0.7822 - val_loss: 3.7667 - val_acc: 0.2800
Epoch 9824/10000
 - 0s - loss: 0.8205 - acc: 0.8200 - val_loss: 3.8452 - val_acc: 0.2800
Epoch 9825/10000
 - 0s - loss: 0.8222 - acc: 0.8000 - val_loss: 3.8383 - val_acc: 0.3000
Epoch 9826/10000
 - 0s - loss: 0.8641 - acc: 0.7778 - val_loss: 3.7440 - val_acc: 0.2600
Epoch 9827/10000
 - 0s - loss: 0.8400 - acc: 0.7978 - val_loss: 3.7669 - val_acc: 0.2400
Epoch 9828/10000
 - 0s - loss: 0.8322 - acc: 0.8244 - val_loss: 3.7440 - val_acc: 0.2800
Epoch 9829/10000
 - 0s - loss: 0.8683 - acc: 0.7978 - val_loss: 3.7410 - val_acc: 0.3000
Epoch 9830/10000
 - 0s - loss: 0.8326 - acc: 0.7956 - val_loss: 3.7185 - val_acc: 0.2800
Epoch 9831/10000
 - 0s - loss: 0.8661 - acc: 0.7911 - val_loss: 3.7454 - val_acc: 0.2800
Epoch 9832/10000
 - 0

 - 0s - loss: 0.8682 - acc: 0.8067 - val_loss: 3.7096 - val_acc: 0.2800
Epoch 9914/10000
 - 0s - loss: 0.8337 - acc: 0.8089 - val_loss: 3.8113 - val_acc: 0.2400
Epoch 9915/10000
 - 0s - loss: 0.8375 - acc: 0.7822 - val_loss: 3.7350 - val_acc: 0.2600
Epoch 9916/10000
 - 0s - loss: 0.8078 - acc: 0.8333 - val_loss: 3.7828 - val_acc: 0.2800
Epoch 9917/10000
 - 0s - loss: 0.8028 - acc: 0.8267 - val_loss: 3.8110 - val_acc: 0.2800
Epoch 9918/10000
 - 0s - loss: 0.8349 - acc: 0.8000 - val_loss: 3.7586 - val_acc: 0.2600
Epoch 9919/10000
 - 0s - loss: 0.8219 - acc: 0.7911 - val_loss: 3.6957 - val_acc: 0.2800
Epoch 9920/10000
 - 0s - loss: 0.8211 - acc: 0.8178 - val_loss: 3.6954 - val_acc: 0.2800
Epoch 9921/10000
 - 0s - loss: 0.8455 - acc: 0.7911 - val_loss: 3.8230 - val_acc: 0.2200
Epoch 9922/10000
 - 0s - loss: 0.7675 - acc: 0.8133 - val_loss: 3.6988 - val_acc: 0.3000
Epoch 9923/10000
 - 0s - loss: 0.8218 - acc: 0.8067 - val_loss: 3.7946 - val_acc: 0.2600
Epoch 9924/10000
 - 0s - loss: 0.8193 

In [76]:
from keras.models import Model
model = Model(inputs=model.inputs, outputs=model.layers[8].output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46_input (InputLayer)  (None, 7)                 0         
_________________________________________________________________
dense_46 (Dense)             (None, 100)               800       
_________________________________________________________________
dropout_37 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_38 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 100)               10100     
_________________________________________________________________
dropout_39 (Dropout)         (None, 100)               0         
__________

In [77]:
train_history =model.fit(x=train_data,
                         y=train_label_onehot,
                         validation_split=0.1, 
                         epochs=10000,
                         batch_size=15,
                         verbose=2)

    #評估準確率
scores = model.evaluate(test_data, test_label_onehot)
print('\n準確率=',scores[1])
acc.append(scores[1])

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
feature_maps = model.predict(test_data)

In [ ]:
# 绘制训练 & 验证的准确率值
plt.figure(figsize=(10, 10))
plt.plot(train_history.history['acc'])
plt.plot(train_history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
plt.figure(figsize=(10, 10))
# 绘制训练 & 验证的损失值
plt.plot(train_history.history['loss'])
plt.plot(train_history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()